<a href="https://colab.research.google.com/github/abhaygangwar3/Ankii-s-Portfolio/blob/master/EfficientNet_to_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('/content/drive/MyDrive/Train.npz')
val_data = np.load('/content/drive/MyDrive/Val.npz')
test_data = np.load('/content/drive/MyDrive/Test.npz')

# Unpack the datasets
x_train, y_train = train_data['images'], train_data['labels']
x_val, y_val = val_data['images'], val_data['labels']
x_test, y_test = test_data['images'], test_data['labels']

# Data normalization (if not already normalized)
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

# Image augmentation
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

# Load EfficientNet model with pre-trained weights, exclude top
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3]))

# Freeze the base model
base_model.trainable = False

# Create the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # For binary classification (gender)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                    epochs=10,
                    validation_data=(x_val, y_val))

# Fine-tune the model (optional)
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=32),
          epochs=5,
          validation_data=(x_val, y_val))

# Evaluate the model
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Female', 'Male'], yticklabels=['Female', 'Male'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# Classification report
print(classification_report(y_test, y_pred, target_names=['Female', 'Male']))
